In [ ]:
import os 

del os.environ["AWS_ACCESS_KEY_ID"]
del os.environ["AWS_SECRET_ACCESS_KEY"]

from dotenv import load_dotenv
# Path to env file containing the waterbodies database credentials
# Only necessary on the Sandbox.
env_path = "/home/jovyan/.env"
load_dotenv(env_path)

In [ ]:
import logging
from odc.geo.geobox import GeoBox
from tqdm import tqdm
from datacube import Datacube
from odc.stats.model import DateTimeRange
from waterbodies.io import check_directory_exists, find_geotiff_files
from waterbodies.logs import logging_setup
from waterbodies.text import parse_tile_id_from_filename
from waterbodies.hopper import create_tasks_from_scenes, GRIDSPEC

In [ ]:
verbose =1
run_type = "gap-filling"
temporal_range = "2024-02--P1M"
historical_extent_rasters_directory = "s3://deafrica-services/waterbodies/v0.0.2/conflux/historical_extent_rasters/"

In [ ]:
# Set up logging.
logging_setup(verbose)
_log = logging.getLogger(__name__)

In [ ]:
# Parse the temporal range
if run_type != "regular-update" :
    temporal_range_ = DateTimeRange(temporal_range)

In [ ]:
if not check_directory_exists(path=historical_extent_rasters_directory):
    e = FileNotFoundError(f"Directory {historical_extent_rasters_directory} does not exist!")
    _log.error(e)
    raise e
else:
    historical_extent_rasters = find_geotiff_files(directory_path=historical_extent_rasters_directory)

In [ ]:
# Get the tile_ids for tiles that actually contain waterbodies.
tile_ids_of_interest = [
    parse_tile_id_from_filename(file_path=raster_file)
    for raster_file in historical_extent_rasters
]

In [ ]:
product = "wofs_ls"

In [ ]:
# Connect to the datacube
dc = Datacube(app="gap-filling")

In [ ]:
# The difference between gap-filling and the other steps is here
# we are searching for datasets by their creation date (`creation_time`),
# not their acquisition date (`time`).
dc_query = dict(product=product, creation_time=(temporal_range_.start, temporal_range_.end))
dc_query

In [ ]:
%%time
# Search the datacube for all wofs_ls datasets whose creation times (not acquisition time)
# fall within the temporal range specified.
# E.g  a dataset can have an aquisition date of 2023-12-15 but have been added to the
# datacube in 2024-02, which will be its creation date.
scenes = dc.find_datasets(**dc_query)

In [ ]:
%%time
# Identify the tasks / waterbody observations affected.
affected_tasks = create_tasks_from_scenes(scenes=scenes, tile_ids_of_interest=tile_ids_of_interest)

In [ ]:
# Get the ids for the affected waterbody observations.
task_ids = [
            task_id for task in affected_tasks for task_id, task_dataset_ids in task.items()
        ]

In [ ]:
# For each task id add an empty list as a place holder for the task datasets ids.
# This will be filled in the drill function which is easier to do in parallel than
# looping over each task to update the required task dataset ids here.
tasks = [{task_id: []} for task_id in task_ids]

In [ ]:
# View the first 100 tasks
tasks[:100]